In [3]:
#Importing Libraries
import pandas as pd
import numpy as np

df = pd.read_csv("./data/Hotel_Reviews.csv")
df= df.drop(['Review_Date','Review_Total_Negative_Word_Counts','Review_Total_Positive_Word_Counts','days_since_review','lat','lng','Additional_Number_of_Scoring','Total_Number_of_Reviews_Reviewer_Has_Given','Positive_Review','Negative_Review','Reviewer_Nationality'], axis=1)

hotelLeisureCounts = df[df['Tags'].str.contains('Leisure')].groupby('Hotel_Name')['Hotel_Name'].count().sort_values(ascending=False)
print "BusinessCounts: ", len(hotelLeisureCounts)
#print(hotelLeisureCounts)

hotelBusinessCounts = df[df['Tags'].str.contains('Business')].groupby('Hotel_Name')['Hotel_Name'].count().sort_values(ascending=False)
print "Leisure Counts: ",len(hotelBusinessCounts)
#print(hotelBusinessCounts)

hotelSoloCounts = df[df['Tags'].str.contains('Solo')].groupby('Hotel_Name')['Hotel_Name'].count().sort_values(ascending=False)
print "Solo Traveller Counts: ",len(hotelSoloCounts)

hotelCoupleCounts = df[df['Tags'].str.contains('Couple')].groupby('Hotel_Name')['Hotel_Name'].count().sort_values(ascending=False)
print "Solo Couple Counts: ",len(hotelCoupleCounts)

hotelGroupCounts = df[df['Tags'].str.contains('Group')].groupby('Hotel_Name')['Hotel_Name'].count().sort_values(ascending=False)
print "Solo Group Counts: ",len(hotelGroupCounts)


BusinessCounts:  1492
Leisure Counts:  1488
Solo Traveller Counts:  1491
Solo Couple Counts:  1492
Solo Group Counts:  1481


In [4]:
#Create a new Dataframe for recommendations
#This will be a binary matrix
processedDF=pd.DataFrame(index=df['Hotel_Name'].unique().tolist())
processedDF['Business']=0;
processedDF['Leisure']=0;
processedDF['Solo']=0;
processedDF['Couple']=0;
processedDF['Group']=0;
#processedDF

In [5]:
uniqueHotels=df['Hotel_Name'].unique().tolist()
for hotel in uniqueHotels:
    #Update Trip type columns
    if hotel in hotelBusinessCounts and hotel in hotelLeisureCounts:
        if hotelLeisureCounts[hotel] > hotelBusinessCounts[hotel]:
            processedDF.at[hotel, 'Leisure'] = 1
        elif hotelLeisureCounts[hotel] < hotelBusinessCounts[hotel]:
            processedDF.at[hotel, 'Business'] = 1
        else:
            processedDF.at[hotel, 'Leisure'] = 1
            processedDF.at[hotel, 'Business'] = 1
    elif hotel in hotelBusinessCounts and hotel not in hotelLeisureCounts:
        processedDF.at[hotel, 'Business'] = 1
    elif hotel in hotelLeisureCounts and hotel not in hotelBusinessCounts:
        #print ("no business tag reviews for: ",hotel)
        processedDF.at[hotel, 'Leisure'] = 1
        
    #Update Traveller type columns
    #print hotelCoupleCounts['Best Western Amiral Hotel']
    if hotel in hotelSoloCounts and hotel in hotelCoupleCounts and hotel in hotelGroupCounts:
        soloCount=hotelSoloCounts[hotel]
        coupleCount=hotelCoupleCounts[hotel]
        groupCount=hotelGroupCounts[hotel]

        if soloCount>coupleCount and soloCount>groupCount:
            processedDF.at[hotel, 'Solo'] = 1
        elif coupleCount>soloCount and coupleCount>groupCount:
            processedDF.at[hotel, 'Couple'] = 1
        elif groupCount>soloCount and groupCount>coupleCount:
            processedDF.at[hotel, 'Group'] = 1
        elif soloCount==coupleCount and soloCount>groupCount:
            processedDF.at[hotel, 'Solo'] = 1
            processedDF.at[hotel, 'Couple'] = 1
        elif soloCount==groupCount and soloCount>coupleCount:
            processedDF.at[hotel, 'Solo'] = 1
            processedDF.at[hotel, 'Group'] = 1
        elif coupleCount==groupCount and coupleCount>soloCount:
            processedDF.at[hotel, 'Couple'] = 1
            processedDF.at[hotel, 'Group'] = 1
        else:
            processedDF.at[hotel, 'Solo'] = 1
            processedDF.at[hotel, 'Couple'] = 1
            processedDF.at[hotel, 'Group'] = 1
    
processedDF

,Business,Leisure,Solo,Couple,Group
Hotel Arena,0,1,0,1,0
K K Hotel George,0,1,0,1,0
Apex Temple Court Hotel,0,1,0,1,0
The Park Grand London Paddington,0,1,0,1,0
Monhotel Lounge SPA,0,1,0,1,0
Kube Hotel Ice Bar,0,1,0,1,0
The Principal London,0,1,0,1,0
Park Plaza County Hall London,0,1,0,1,0
One Aldwych,0,1,0,1,0
Splendid Etoile,0,1,0,1,0


In [6]:
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

userDF= pd.DataFrame(index=['user'],columns=['Business','Leisure','Solo','Couple','Group'])
userDF.loc['user'] = [0,1,0,1,0]
userDF

,Business,Leisure,Solo,Couple,Group
user,0,1,0,1,0


In [7]:
similarityDF=cosine_similarity(processedDF,userDF)
#len(similarity.tolist())
#similarity.tolist()
similarityDF = pd.DataFrame(similarityDF)
#Update similarity dataframe to add the average score column
similarityDF['Average_Score']=0.0;

for index, row in similarityDF.iterrows():
    #Get the hotel row from original dataframe
    hotelRow= df.loc[df['Hotel_Name'] == processedDF.index[index]].head(1)
    #Update score column
    similarityDF.at[index, 'Average_Score'] = hotelRow['Average_Score']
#print similarityDF
#Sort similarityDF by cosine similarity score and then hotel average score
similarityDF=similarityDF.sort_values(by=[0,'Average_Score'],ascending=False).head(5)
for index, row in similarityDF.iterrows():
    print "Hotel Name: ", processedDF.index[index]
    hotelRow= df.loc[df['Hotel_Name'] == processedDF.index[index]].head(1)
    print "Average Score: ", hotelRow['Average_Score']
    print processedDF.iloc[index]
    print 'Similarity score',row[0]
    print '\n'


Hotel Name:  Haymarket Hotel
Average Score:  14708    9.6
Name: Average_Score, dtype: float64
Business    0
Leisure     1
Solo        0
Couple      1
Group       0
Name: Haymarket Hotel, dtype: int64
Similarity score 1.0


Hotel Name:  H tel de La Tamise Esprit de France
Average Score:  176997    9.6
Name: Average_Score, dtype: float64
Business    0
Leisure     1
Solo        0
Couple      1
Group       0
Name: H tel de La Tamise Esprit de France, dtype: int64
Similarity score 1.0


Hotel Name:  41
Average Score:  185602    9.6
Name: Average_Score, dtype: float64
Business    0
Leisure     1
Solo        0
Couple      1
Group       0
Name: 41, dtype: int64
Similarity score 1.0


Hotel Name:  Hotel Casa Camper
Average Score:  316447    9.6
Name: Average_Score, dtype: float64
Business    0
Leisure     1
Solo        0
Couple      1
Group       0
Name: Hotel Casa Camper, dtype: int64
Similarity score 1.0


Hotel Name:  Hotel The Serras
Average Score:  398945    9.6
Name: Average_Score, dtype: